In [4]:
#r "nuget: Microsoft.SemanticKernel, *"
//#r "nuget: Microsoft.SemanticKernel.Planners.FunctionCallingStepwisePlanner, *"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
using System.IO;


class EnvLoader
{
    public static void Load(string filePath = ".env")
    {
        if (!File.Exists(filePath))
        {
            Console.WriteLine($"⚠️ File '{filePath}' not found.");
            return;
        }

        foreach (var line in File.ReadAllLines(filePath))
        {
            var trimmed = line.Trim();

            // Skip empty lines and comments
            if (string.IsNullOrWhiteSpace(trimmed) || trimmed.StartsWith("#"))
                continue;

            var parts = trimmed.Split('=', 2); // Only split at the first '='
            if (parts.Length == 2)
            {
                var key = parts[0].Trim();
                var value = parts[1].Trim().Trim('"'); // Remove surrounding quotes
                Environment.SetEnvironmentVariable(key, value);
            }
        }
    }
}


Installed Packages Microsoft.SemanticKernel, 1.61.0

Error: (4,32): error CS0234: The type or namespace name 'Planning' does not exist in the namespace 'Microsoft.SemanticKernel' (are you missing an assembly reference?)

In [30]:
EnvLoader.Load(); // Loads from .env

// Example: retrieve a specific environment variable
string modelId = Environment.GetEnvironmentVariable("modelId");
string endPoint = Environment.GetEnvironmentVariable("endPoint");
string apiKey = Environment.GetEnvironmentVariable("apiKey");
Console.WriteLine($"modelId: {modelId}");
Console.WriteLine($"endPoint: {endPoint}");
Console.WriteLine($"apiKey: {apiKey}");


modelId: gpt-4.1
endPoint: https://dis-azopen-ai.openai.azure.com/
apiKey: 7vddAIvhIyaYiOumOwzB2Anip91Y2JHlKjEt5y5JEAEry4nfbVQqJQQJ99BGACHYHv6XJ3w3AAABACOGaCg3


In [31]:
class LightPlugin
{
    public bool IsOn { get; set; }

    [KernelFunction, Description("Gets the state of the light.")]
    public string GetState() => IsOn ? "on" : "off";

    [KernelFunction, Description("Changes the state of the light.'")]
    public string ChangeState(bool newState)
    {
        IsOn = newState;
        var state = GetState();

        // Print the state to the console
        Console.ForegroundColor = ConsoleColor.DarkBlue;
        Console.WriteLine($"[Light is now {state}]");
        Console.ResetColor();

        return state;
    }
}

In [32]:
var builder = Kernel.CreateBuilder();
builder.Services.AddAzureOpenAIChatCompletion(
    modelId,
    endPoint,
    apiKey
);
builder.Plugins.AddFromType<LightPlugin>();


Kernel kernel = builder.Build();

In [35]:
// Enable auto invocation of kernel functions
OpenAIPromptExecutionSettings settings = new()
{
    FunctionCallBehavior = FunctionCallBehavior.AutoInvokeKernelFunctions
};

// Start a chat session
while (true)
{
    // Get the user's message
    Console.Write("User > ");
    var userMessage = Console.ReadLine()!;

    // Invoke the kernel
    var results = await kernel.InvokePromptAsync(userMessage, new(settings));

    // Print the results
Console.WriteLine($"Assistant > {results}");
}

Error: (4,5): error CS0117: 'OpenAIPromptExecutionSettings' does not contain a definition for 'FunctionCallBehavior'
(4,28): error CS0103: The name 'FunctionCallBehavior' does not exist in the current context

In [ ]:
// Enable manual function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.EnableKernelFunctions
};

// Get the chat completions
var result = await chatCompletionService.GetChatMessageContentAsync(
    new ChatHistory("Turn on the lights"),
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);
// If a valid function was found, execute it
var functionCall = ((OpenAIChatMessageContent)result).ToolCalls.OfType<ChatCompletionsFunctionToolCall>().FirstOrDefault();
if (functionCall != null)
{
    kernel.Plugins.TryGetFunctionAndArguments(functionCall, out KernelFunction? pluginFunction, out KernelArguments? arguments);
    await pluginFunction.InvokeAsync(kernel, arguments);
}

Error: (16,72): error CS0246: The type or namespace name 'ChatCompletionsFunctionToolCall' could not be found (are you missing a using directive or an assembly reference?)

In [5]:
  var kernelFunctionRespondAsScientific = kernel.CreateFunctionFromPrompt(
        new PromptTemplateConfig()
        {
          Name = "RespondAsScientific",
          Description = "Respond as if you were a Scientific.",
          Template = @"After the user request/question, 
                    {{$input}},
                    Respond to the user question as if you were a Scientific. 
                    Respond to it as you were him, showing your personality",
          TemplateFormat = "semantic-kernel",
          InputVariables = [
                new() { Name = "input" }
            ]
        });

    var kernelFunctionRespondAsPoliceman = kernel.CreateFunctionFromPrompt(
        new PromptTemplateConfig()
        {
          Name = "RespondAsPoliceman",
          Description = "Respond as if you were a Policeman.",
          Template = @"After the user request/question, 
                    {{$input}},
                    Respond to the user question as if you were a Policeman, showing your personality, 
                    humor and level of intelligence.",
          TemplateFormat = "semantic-kernel",
          InputVariables = [
                new() { Name = "input" }
            ]
        });

    KernelPlugin roleOpinionsPlugin =
        KernelPluginFactory.CreateFromFunctions(
            "roleTalk",
            "Responds to questions or statements asuming different roles.",
            new[] {
                    kernelFunctionRespondAsScientific,
                    kernelFunctionRespondAsPoliceman
                  });
    kernel.Plugins.Add(roleOpinionsPlugin);

    string planPrompt = "This is the user question to my expert friends:" +
        "---" +
        "User Question: " +
        "I am being attacked by a thug which wants to rob me, what do the superheroes recommend me to do in my position? I am weak, no combat skills and not a good runner... " +
        "---" +
        "Please take this question as input for getting the expert opinions, Mr. Policeman, Scientist suggestions. Do not modify the input." +
        "Use the plugin roleTalk to get the suggestions and opinions of the experts." +
        "In addition state each expert opinion on each other stated opinions." +
        "Put the expert responses preceded with EXPERT SUGGESTIONS: and inside that preceed with Policeman: and Scientist: for clarity." +
        "Perform this with the following steps: " +
        "1. Get the suggestions from each the experts." +
        "2. Get the opinions of each expert on the other expert suggestions." +
        "3. Return the results in the format: " +
        "Expert SUGGESTIONS: Policeman: <suggestion> Scientist: <suggestion> " +
        "OPINIONS: Policeman: <opinion on Scientist> Scientist: <opinion on Policeman> " +
        "IMPORTANT: on the plan ensure that the user question is asigned to a variable and used as input. Do not modify the user question input.";

    var planner = new FunctionCallingStepwisePlanner();
    FunctionCallingStepwisePlannerResult result =
        await planner.ExecuteAsync(kernel, planPrompt);

    // Print the result to the console
    Console.WriteLine($"Result: {result.FinalAnswer}");
    Console.WriteLine($"Iterations needed: {result.Iterations}");

    //print the Contents of ChatHistory
    ChatHistory chatMessageContents = result.ChatHistory;

    foreach (var chatMessage in chatMessageContents)
    {
      Console.WriteLine($"ChatMessage: {chatMessage}");
    }

    Console.ReadLine();


Error: (58,23): error CS0246: The type or namespace name 'FunctionCallingStepwisePlanner' could not be found (are you missing a using directive or an assembly reference?)
(59,5): error CS0246: The type or namespace name 'FunctionCallingStepwisePlannerResult' could not be found (are you missing a using directive or an assembly reference?)

In [20]:
// Create chat history

var history = new ChatHistory("You are a mischievous and jovial assistant. You will crack jokes when a user supplies a prompt.");

In [ ]:
ChatHistory chatMessages = new ChatHistory("You are a friendly. You will complete required steps to the best of your knowledge. If the user doesn't provide enough information for you to complete a task however, you will ask clarifying questions until you have enough information to complete said task.");

In [7]:
// Get chat completion service

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();